In [1]:
!pip install selenium
!pip install undetected-chromedriver


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [1]:
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import undetected_chromedriver as uc
import pandas as pd
import time
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [50]:
squares = []
layouts = []
types_uch = []
links=[]
addresses = []
main_prices = []
sq_prices = []
descriptions = []
lost_descriptions = []
broken_title = []

In [54]:
# парсинг по листам объявлений
start_time = time.time()
url = "https://krym.cian.ru/kupit-zemelniy-uchastok-bez-posrednikov/"

driver = uc.Chrome()
driver.get(url)

cookie_notification = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, ".//div[@data-name='CookiesNotification']"))
)

# Закрыть уведомление о файлах cookie, если оно видимо
if cookie_notification.is_displayed():
    cookie_button = cookie_notification.find_element(By.CLASS_NAME, "_25d45facb5--button--KVooB._25d45facb5--button--gs5R_._25d45facb5--M--I5Xj6._25d45facb5--button--DsA7r")
    cookie_button.click()

# Переключиться на основное окно
driver.switch_to.default_content()
while True:
    time.sleep(4)
    # Извлечение данных объявлений
    ads = driver.find_elements(By.XPATH, ".//article[@data-name='CardComponent']")
    for ad in ads:
        link = ad.find_element(By.CLASS_NAME, "_93444fe79c--link--eoxce").get_attribute('href')
        if link in links:
            continue
        title = ad.find_element(By.XPATH, ".//span[@data-mark='OfferTitle']").text
        try:

            layout = title[:title.index(',')].strip()
            try:
                square = float(title[title.index(',')+1:title.index('сот.')].strip().replace(' ', '').replace(',', '.'))*100
            except AttributeError:
                print(title)
                broken_title.append(link)
                continue
            if square == '':
                broken_title.append(link)
                continue
        except Exception as e:
            print(title)
            broken_title.append(link)
            continue
        layouts.append(layout)
        squares.append(square)
        links.append(ad.find_element(By.CLASS_NAME, "_93444fe79c--link--eoxce").get_attribute('href'))

        addresses.append(ad.find_element(By.CLASS_NAME, "_93444fe79c--labels--L8WyJ").text)

        main_prices.append(ad.find_element(By.XPATH, ".//span[@data-mark='MainPrice']").text)
        sq_prices.append(float(ad.find_element(By.XPATH, ".//span[@data-mark='MainPrice']").text.replace(' ', '').replace(',', '.')[:-1])/square)
        try:
            descriptions.append(ad.find_element(By.CLASS_NAME, "_93444fe79c--description--SqTNp").text)
        except Exception as e:
            descriptions.append('')
            lost_descriptions.append(len(addresses))
    # Проверка наличия кнопки "Дальше"
    try:
        next_button = driver.find_element(By.XPATH, '//a[contains(@class, "_93444fe79c--button--Cp1dl") and contains(@class, "_93444fe79c--link-button--Pewgf") and contains(@class, "_93444fe79c--M--T3GjF") and contains(@class, "_93444fe79c--button--dh5GL") and span[@class="_93444fe79c--text--rH6sj" and text()="Дальше"]]')
    except Exception as e:
        print(e)
        break
    # Кликнуть на кнопку "Дальше" для перехода на следующую страницу
    next_button.click()
print(len(squares),len(layouts),len(types_uch),len(addresses),len(main_prices),len(sq_prices),len(descriptions))
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

time.sleep(1)

driver.execute_script("window.scrollTo(0, -250);")
time.sleep(1)

while True:
    try:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)
        next_button2 = driver.find_element(By.CLASS_NAME, '_93444fe79c--moreSuggestionsButtonContainer--h0z5t')
        driver.execute_script("window.scrollTo(0, -250);")
        time.sleep(1)
    except Exception as e:
        print(e)
        break
    next_button2.click()

ads = driver.find_elements(By.XPATH, ".//article[@data-name='CardComponent']")
for ad in ads:
    link = ad.find_element(By.CLASS_NAME, "_93444fe79c--link--eoxce").get_attribute('href')
    if link in links:
        continue
    title = ad.find_element(By.XPATH, ".//span[@data-mark='OfferTitle']").text
    try:

        layout = title[:title.index(',')].strip()
        try:
            square = float(title[title.index(',')+1:title.index('сот.')].strip().replace(' ', '').replace(',', '.'))*100
        except AttributeError:
            print(title)
            broken_title.append(link)
            continue
        if square == '':
            broken_title.append(link)
            continue
    except:
        link = ad.find_element(By.CLASS_NAME, "_93444fe79c--link--eoxce").get_attribute('href')
        print(title)
        broken_title.append(link)
        continue
    layouts.append(layout)
    squares.append(square)
    links.append(ad.find_element(By.CLASS_NAME, "_93444fe79c--link--eoxce").get_attribute('href'))

    addresses.append(ad.find_element(By.CLASS_NAME, "_93444fe79c--labels--L8WyJ").text)

    main_prices.append(ad.find_element(By.XPATH, ".//span[@data-mark='MainPrice']").text)

    sq_prices.append(float(ad.find_element(By.XPATH, ".//span[@data-mark='MainPrice']").text.replace(' ', '').replace(',', '.')[:-1])/square)

    try:
        descriptions.append(ad.find_element(By.CLASS_NAME, "_93444fe79c--description--SqTNp").text)
    except NoSuchElementException:
        descriptions.append('')
        lost_descriptions.append(len(addresses))
len(broken_title)
print(len(links),len(squares),len(main_prices),len(sq_prices),len(addresses),len(descriptions))

Участок в СНТ "Мечта-1"
УчастокИЖС
Участок, 2 га, Фермерское хозяйство
Земля возле моря.
Участок, 1.37 га, Фермерское хозяйство
5 минут от города
Участок, 1.14 га, Фермерское хозяйство
Земельный участок в Крыму
Участок, 3 га, Фермерское хозяйство
Участок, 6 га, Садоводство
Участок, 1 га
Участок, 7.2 га, Фермерское хозяйство
Участок, 10 га
Участок, 2.27 га, Фермерское хозяйство
Участок, 2.97 га, Фермерское хозяйство
Участок, 1.5 га
Участок, 1.56 га
Участок, 6.1 га, ИЖС
Продам видовой земельный участок
Участок, 2 га
Участок, 3 га
Участок под бизнес на ЮБК
Участок в СНД Здоровье. Феодосия.
Участок, 3 га, Фермерское хозяйство
Собственник!
Продается участок 100 соток лпх
Участок, 20 га, ДНП
Участок, 4.48 га
Участок, 3.5 га, Фермерское хозяйство
Участок, 1.9 га
Участок, 1 га, Фермерское хозяйство
Участок, 3 га, Садоводство
Участок, 20.1 га, ДНП
Участок, 5.6 га
Участок, 7 га, Садоводство
Участок, 0.75 га, ИЖС
Участок, 1 га
Участок, 2 га, Фермерское хозяйство
Участок, 30 га
Участок, 3.46 га
Уч

In [55]:
print(len(links),len(squares),len(main_prices),len(sq_prices),len(addresses),len(descriptions))
data = {
    'Ссылка' : links,
    'Площадь' : squares,
    'Стоимость' : main_prices,
    'Цена за кв. м.': sq_prices,
    'Адрес': addresses,
    'Описание': descriptions,
}
driver.quit()
end_time = time.time()
execution_time = end_time - start_time
print(f"{len(squares)} объявлений обработаны за {execution_time} секунд.")
df = pd.DataFrame(data)
df

4102 4102 4102 4102 4102 4102
4102 объявлений обработаны за 429.0597195625305 секунд.


,Ссылка,Площадь,Стоимость,Цена за кв. м.,Адрес,Описание
0,https://krym.cian.ru/sale/suburban/279455189/,450.0,1 750 000 ₽,3888.888889,"Республика Крым, Алушта городской округ, с. Пр...",Объект 40697 Продается земельный участок с вид...
1,https://krym.cian.ru/sale/suburban/269686050/,800.0,1 500 000 ₽,1875.000000,"Республика Крым, Алушта городской округ, с. Из...",Объект 25447 Мы находим уникальные варианты и ...
2,https://krym.cian.ru/sale/suburban/260920743/,600.0,490 000 ₽,816.666667,"Республика Крым, Феодосия городской округ, При...",СРОЧНО! Очень доступный по цене участок с полн...
3,https://krym.cian.ru/sale/suburban/285451554/,900.0,900 000 ₽,1000.000000,"Республика Крым, Феодосия, переулок Корабельны...",Участок площадью 9 соток на мысе Ильи. Коммуни...
4,https://krym.cian.ru/sale/suburban/272279033/,900.0,1 300 000 ₽,1444.444444,"Республика Крым, Алушта городской округ, с. Ма...",Объект 28843 Мы находим уникальные варианты и ...
...,...,...,...,...,...,...
4097,https://krym.cian.ru/sale/suburban/287808428/,428.0,7 000 000 ₽,16355.140187,"Республика Крым, Симферополь, р-н Центральный,...",Продается участок 428 кв.м. в хорошем месте. В...
4098,https://krym.cian.ru/sale/suburban/287506129/,700.0,7 900 000 ₽,11285.714286,"Республика Крым, Ялта городской округ, пос. Ли...",Срочная продажа. Большая Ялта. Возле Гурзуфа. ...
4099,https://krym.cian.ru/sale/suburban/287855819/,1708.0,12 000 000 ₽,7025.761124,"Республика Крым, Черноморский район, Оленевско...",90:14:110101:1170 17.08 соток ИЖС Участок ровн...
4100,https://krym.cian.ru/sale/suburban/287880402/,16525.0,15 000 000 ₽,907.715582,"Республика Крым, Бахчисарайский район, с. Песч...",Собственник. Продаётся участок сельхозназначен...


In [56]:
df.to_csv('Крым_участок_циан_230609.csv', index=False)

In [16]:
# парсинг по каждому объявлению
latitude = []
longitude = []
dates = []
total_area = []
addresses = []
main_prices = []
sq_prices = []
descriptions = []


start_time = time.time()
driver = uc.Chrome()
i = 200
links1 = links + broken_title
print(len(links1))
for link in links+broken_title:
    try:
        driver.get(link)
        # latitude =
        # longitude =

        dates.append(driver.find_element(By.CLASS_NAME, "a10a3f92e9--color_gray40_100--ppbi0.a10a3f92e9--lineHeight_5u--cJ35s.a10a3f92e9--fontWeight_normal--P9Ylg.a10a3f92e9--fontSize_14px--TCfeJ.a10a3f92e9--display_block--pDAEx.a10a3f92e9--text--g9xAG.a10a3f92e9--text_letterSpacing__0--mdnqq").text)


        data1 = driver.find_elements(By.XPATH, ".//div[@data-name='ObjectFactoidsItem']")

        total_area.append(data1[0].text.split('\n')[1])


        addresses.append(driver.find_element(By.CLASS_NAME, "a10a3f92e9--address-line--GRDTb").text[:-8])
        main_prices.append(driver.find_element(By.XPATH, ".//div[@data-name='PriceInfo']").text)
        data2 = driver.find_elements(By.CLASS_NAME, "a10a3f92e9--color_black_100--kPHhJ.a10a3f92e9--lineHeight_20px--tUURJ.a10a3f92e9--fontWeight_normal--P9Ylg.a10a3f92e9--fontSize_14px--TCfeJ.a10a3f92e9--display_block--pDAEx.a10a3f92e9--text--g9xAG.a10a3f92e9--text_letterSpacing__normal--xbqP6")
        sq_prices.append(data2[1].text)
        descriptions.append(driver.find_element(By.CLASS_NAME, "a10a3f92e9--color_black_100--kPHhJ.a10a3f92e9--lineHeight_6u--A1GMI.a10a3f92e9--fontWeight_normal--P9Ylg.a10a3f92e9--fontSize_16px--RB9YW.a10a3f92e9--display_block--pDAEx.a10a3f92e9--text--g9xAG.a10a3f92e9--text_letterSpacing__0--mdnqq.a10a3f92e9--text_whiteSpace__pre-wrap--scZwb").text)
    except Exception as e:
        print()
        print(e)
        print(link)
    i-=1
    if i ==0:
        break
    if i %10 == 0:
        print(i)
driver.quit()


1537
190
180
170
160
150
140
130
120
110
100
90
80
70
60
50
40
30
20
10


In [17]:
print(len(dates))
data = {
    # 'latitude': latitude,
    # 'longitude': longitude,
    'dates' : dates,
    'total_area' : total_area,
    'addresses' : addresses,
    'main_prices' : main_prices,
    'sq_prices' : sq_prices,
    'descriptions' : descriptions,
}
driver.quit()
end_time = time.time()
execution_time = end_time - start_time
print(f"{len(dates)} объявлений обработаны за {execution_time} секунд.")
df = pd.DataFrame(data)
df

200
200 объявлений обработаны за 379.5182251930237 секунд.


,dates,total_area,addresses,main_prices,sq_prices,descriptions
0,"Обновлено: 5 июн, 18:16","197,09 м²","Крым респ., Алушта, Профессорский Уголок микро...",40 032 527 ₽,203 118 ₽/м²,Продаётся 3-комнатная квартира в сданном доме ...
1,"Обновлено: 5 июн, 18:46","48,39 м²","Крым респ., Алушта, Профессорский Уголок микро...",9 529 946 ₽,196 940 ₽/м²,Продаётся 2-комнатная квартира в строящемся до...
2,"Обновлено: 23 май, 11:06",25 м²,"Крым респ., Бахчисарайский район, Песчаное сел...",1 800 000 ₽,72 000 ₽/м²,"Апартаменты в цоколе 25 м2, трехэтажного дома...."
3,"Обновлено: 5 июн, 18:46","60,71 м²","Крым респ., Алушта, ул. Западная, 4к3",8 900 000 ₽,146 599 ₽/м²,Продаются 1-комнатные апартаменты в сданном до...
4,"Обновлено: 23 май, 11:07","65,1 м²","Крым респ., Бахчисарайский район, Песчаное сел...",5 300 000 ₽,81 413 ₽/м²,"Трехкомнатная квартира 65,1 м2 на этаже 2/2 до..."
...,...,...,...,...,...,...
195,"Обновлено: 3 июн, 04:54",32 м²,"Крым респ., Симферополь, р-н Железнодорожный, ...",4 200 000 ₽,131 250 ₽/м²,Лучшее решение для вас в одном из самых привле...
196,"Обновлено: сегодня, 05:19",130 м²,"Крым респ., Ялта городской округ, Гурзуф пгт, ...",31 000 000 ₽,238 462 ₽/м²,Квартира 801 на 9 этаже комплекса с собственно...
197,"Обновлено: 30 май, 14:59",20 м²,"Крым респ., Алушта, пер. Базарный, 1",3 700 000 ₽,185 000 ₽/м²,МЫ НЕ БЕРЕМ КОМИССИЮ С ПОКУПАТЕЛЯ!!!\nПредлага...
198,"Обновлено: сегодня, 05:05","87,17 м²","Крым респ., Симферопольский район, Мирное село...",8 542 660 ₽,98 000 ₽/м²,Продаётся 3-комнатная квартира в строящемся до...


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

dates = []
total_area = []
addresses = []
main_prices = []
sq_prices = []
descriptions = []

start_time = time.time()

for link in links:
    try:
        response = requests.get(link)
        soup = BeautifulSoup(response.content, 'html.parser')

        dates.append(soup.find('div', class_='a10a3f92e9--color_gray40_100--ppbi0.a10a3f92e9--lineHeight_5u--cJ35s'
                                             '.a10a3f92e9--fontWeight_normal--P9Ylg.a10a3f92e9--fontSize_14px--TCfeJ'
                                             '.a10a3f92e9--display_block--pDAEx.a10a3f92e9--text--g9xAG.a10a3f92e9'
                                             '--text_letterSpacing__0--mdnqq').text)

        data1 = soup.find_all('div', {'data-name': 'ObjectFactoidsItem'})
        total_area.append(data1[0].find_all('span')[1].text)

        addresses.append(soup.find('div', class_='a10a3f92e9--address-line--GRDTb').text[:-8])
        main_prices.append(soup.find('div', {'data-name': 'PriceInfo'}).text)

        data2 = soup.find_all('span', class_='a10a3f92e9--color_black_100--kPHhJ a10a3f92e9--lineHeight_20px--tUURJ '
                                             'a10a3f92e9--fontWeight_normal--P9Ylg a10a3f92e9--fontSize_14px--TCfeJ '
                                             'a10a3f92e9--display_block--pDAEx a10a3f92e9--text--g9xAG '
                                             'a10a3f92e9--text_letterSpacing__normal--xbqP6')
        sq_prices.append(data2[1].text)

        descriptions.append(soup.find('span', class_='a10a3f92e9--color_black_100--kPHhJ '
                                                     'a10a3f92e9--lineHeight_6u--A1GMI '
                                                     'a10a3f92e9--fontWeight_normal--P9Ylg '
                                                     'a10a3f92e9--fontSize_16px--RB9YW '
                                                     'a10a3f92e9--display_block--pDAEx a10a3f92e9--text--g9xAG '
                                                     'a10a3f92e9--text_letterSpacing__0--mdnqq '
                                                     'a10a3f92e9--text_whiteSpace__pre-wrap--scZwb').text)
    except Exception as e:
        print()
        print(e)
        print(link)

In [ ]:
data = {
    'dates': dates,
    'total_area': total_area,
    'addresses': addresses,
    'main_prices': main_prices,
    'sq_prices': sq_prices,
    'descriptions': descriptions,
}

end_time = time.time()
execution_time = end_time - start_time
print(f"{len(dates)} объявлений обработаны за {execution_time} секунд.")

df = pd.DataFrame(data)
print(df)